In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

# Load the environment variable containing the OneMap token
load_dotenv('key.env')
TOKEN = os.getenv('ONEMAPTOKEN')

%run get_bus_info_function.ipynb
%run get_geospatial_function.ipynb
bus_services_df = get_bus_info("https://datamall2.mytransport.sg/ltaodataservice/BusServices", api_key)
bus_routes_df = get_bus_info("https://datamall2.mytransport.sg/ltaodataservice/BusRoutes", api_key)
bus_stops_df = get_bus_info("https://datamall2.mytransport.sg/ltaodataservice/BusStops", api_key)
geospatial_train_path = "../datasets/geospatial_layer/TrainStation_Jul2024/RapidTransitSystemStation.shp"
train_stations = pd.read_excel("../datasets/Train_Stations.xls")
geospatial_train_gdf = gpd.read_file(geospatial_train_path)

Exception: File `'get_bus_info_function.ipynb'` not found.

In [ ]:
# Define the function to fetch route data from OneMap
def fetch_onemap_route(service_no, direction):
    url = 'https://developers.onemap.sg/publicapi/busexp/getOneBusRoute'
    params = {
        'busNo': service_no,
        'direction': direction,
        'token': TOKEN
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data for bus service {service_no} direction {direction}")
        return None

# Process each row in bus_route_df to retrieve and save routes
for _, row in bus_routes_df.iterrows():
    service_no = row['ServiceNo']
    direction = row['Direction']
    result = fetch_onemap_route(service_no, direction)
    if result:
        # Save to a JSON file or further process as needed
        filename = f"../datasets/routes/{service_no}_direction_{direction}.json"
        with open(filename, 'w') as file:
            json.dump(result, file, indent=2)
        print(f"Route data saved for service {service_no}, direction {direction}")

# Example usage
fetch_onemap_route('10', 1)
